In [ ]:
faq_knowledge_base = {
    # Capitals of ~30 countries
    "what is the capital of afghanistan": "The capital of Afghanistan is Kabul.",
    "what is the capital of australia": "The capital of Australia is Canberra.",
    "what is the capital of brazil": "The capital of Brazil is Brasília.",
    "what is the capital of canada": "The capital of Canada is Ottawa.",
    "what is the capital of china": "The capital of China is Beijing.",
    "what is the capital of france": "The capital of France is Paris.",
    "what is the capital of germany": "The capital of Germany is Berlin.",
    "what is the capital of india": "The capital of India is New Delhi.",
    "what is the capital of indonesia": "The capital of Indonesia is Jakarta.",
    "what is the capital of italy": "The capital of Italy is Rome.",
    "what is the capital of japan": "The capital of Japan is Tokyo.",
    "what is the capital of mexico": "The capital of Mexico is Mexico City.",
    "what is the capital of nepal": "The capital of Nepal is Kathmandu.",
    "what is the capital of netherlands": "The capital of the Netherlands is Amsterdam.",
    "what is the capital of pakistan": "The capital of Pakistan is Islamabad.",
    "what is the capital of russia": "The capital of Russia is Moscow.",
    "what is the capital of saudi arabia": "The capital of Saudi Arabia is Riyadh.",
    "what is the capital of south africa": "South Africa has three capitals: Pretoria, Bloemfontein, and Cape Town.",
    "what is the capital of south korea": "The capital of South Korea is Seoul.",
    "what is the capital of spain": "The capital of Spain is Madrid.",
    "what is the capital of sri lanka": "The capital of Sri Lanka is Sri Jayawardenepura Kotte.",
    "what is the capital of thailand": "The capital of Thailand is Bangkok.",
    "what is the capital of turkey": "The capital of Turkey is Ankara.",
    "what is the capital of ukraine": "The capital of Ukraine is Kyiv.",
    "what is the capital of united arab emirates": "The capital of UAE is Abu Dhabi.",
    "what is the capital of united kingdom": "The capital of the United Kingdom is London.",
    "what is the capital of united states": "The capital of the United States is Washington, D.C.",
    "what is the capital of vietnam": "The capital of Vietnam is Hanoi.",
    "what is the capital of zimbabwe": "The capital of Zimbabwe is Harare.",

    # India and Neighbouring Countries - Directions and capitals
    "which countries are in the north of india": "To the north of India are China, Nepal and Bhutan.",
    "which countries are in the northwest of india": "To the northwest of India are Pakistan and Afghanistan.",
    "which countries are in the east of india": "To the east of India are Bangladesh and Myanmar.",
    "which countries are in the south east of india": "The southeastern neighbours of India include Myanmar and Indonesia.",
    "which countries are in the west of india": "To the west of India are Pakistan and Afghanistan.",
    "which countries share land borders with india": "India shares land borders with Afghanistan, Bangladesh, Bhutan, China, Myanmar, Nepal, Pakistan, and Sri Lanka.",
    "which countries share maritime borders with india": "India shares maritime borders with Sri Lanka and the Maldives.",
    "what is the capital of nepal": "The capital of Nepal is Kathmandu.",
    "what is the capital of bhutan": "The capital of Bhutan is Thimphu.",
    "what is the capital of bangladesh": "The capital of Bangladesh is Dhaka.",
    "what is the capital of myanmar": "The capital of Myanmar is Naypyidaw.",
    "what is the capital of pakistan": "The capital of Pakistan is Islamabad.",
    "what is the capital of sri lanka": "The capital of Sri Lanka is Sri Jayawardenepura Kotte.",

    # Additional info
    "what ocean is south of india": "The Indian Ocean lies to the south of India.",
    "which seas surround india": "The Arabian Sea is to the west and the Bay of Bengal is to the east of India.",
    "how many neighbours does india have": "India has 7 land neighbours and 2 maritime neighbours making 9 in total.",
}


In [1]:
# Chatbot using Hugging Face Transformers
# ----------------------------------------
# This notebook implements a simple conversational chatbot using a small local model.
# It maintains a short-term memory of the conversation and supports a CLI-style flow.

!pip install transformers torch --quiet


In [2]:
# Import dependencies
from transformers import pipeline


In [10]:
# Load model (small model for local use)
def load_model(model_name="distilgpt2"):
    print(f"Loading model: {model_name} ...")
    generator = pipeline(
        "text-generation",
        model=model_name,
        pad_token_id=50256  # EOS token for GPT2 padding
    )
    return generator

generator = load_model()


Loading model: distilgpt2 ...


Device set to use cpu


In [13]:
class ChatMemory:
    def __init__(self, max_turns=3):
        self.max_turns = max_turns
        self.history = []

    def update(self, user_input, bot_response):
        self.history.append(f"User: {user_input}\nBot: {bot_response}")
        if len(self.history) > self.max_turns:
            self.history = self.history[-self.max_turns:]

    def get_context(self):
        return "\n".join(self.history)

memory = ChatMemory()

In [12]:
# Initialize memory
# memory = ChatMemory(max_turns=5)


In [14]:
def chatbot_loop():
    print("Chatbot ready! Type '/exit' to quit.\n")
    while True:
        user_input = input("User: ").strip()
        if user_input.lower() == '/exit':
            print("Exiting chatbot. Goodbye!")
            break

        # Prepare prompt from recent conversation history plus new user input
        prompt = memory.get_context() + f"\nUser: {user_input}\nBot:"

        # Generate response with truncation enabled and max_new_tokens only
        response = generator(
            prompt,
            max_new_tokens=150,
            truncation=True,
            do_sample=True,
            temperature=0.7,
            pad_token_id=50256
        )

        # Extract bot reply text cleanly
        bot_response = response[0]['generated_text'].split("Bot:")[-1].split("User:")[0].strip()

        print(f"Bot: {bot_response}")

        # Update conversation memory
        memory.update(user_input, bot_response)

In [15]:
# Run chatbot (works like a CLI inside Jupyter)
chatbot_loop()


Chatbot ready! Type '/exit' to quit.

User: Hello
Bot: Hello
User: How are you?
Bot: Hello
User: What is the capital of France?
Bot: I will be using this system for a new
User: What is the capital of France?
Bot: I will be using this system for a new
User: And what about Italy?
Bot: I will be using this system for a new
User: /exit
Exiting chatbot. Goodbye!


In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Initialize chat history
chat_history_ids = None

print("Chatbot ready! Type '/exit' to quit.\n")

while True:
    user_input = input("User: ").strip()
    if user_input.lower() == "/exit":
        print("Exiting chatbot. Goodbye!")
        break

    # Encode user input with EOS token
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Concatenate with previous history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids is not None else new_user_input_ids

    # Generate reply with attention_mask passed explicitly
    response = model.generate(
        bot_input_ids,
        max_length=150,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        attention_mask=torch.ones_like(bot_input_ids)
    )

    # Decode response, remove prompt part
    bot_reply = tokenizer.decode(response[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    print("Bot:", bot_reply)

    # Update chat history
    chat_history_ids = response


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Chatbot ready! Type '/exit' to quit.

User: Hello
Bot: Hi!
User: What is the capital of France?
Bot: I don't know
User: And what about Italy?
Bot: It has been long since the last time I heard of it
User: so you don't know about italy?
Bot: I do, but I just don't know the capital of italy
User: ok get it
Bot: I already know the capital of italy
User: say what is it?
Bot: I don't know, probably
User: but you said that you already know
Bot: I don't know
User: ok ok thank you
Bot: You're on the list
User: he he he
Bot: oh
User: ok bye
Bot: I'll be on the list
User: /exit
Exiting chatbot. Goodbye!


In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# --- FAQ Knowledge Base ---
faq_knowledge_base = {
    # Capitals of ~30 countries
    "what is the capital of afghanistan": "The capital of Afghanistan is Kabul.",
    "what is the capital of australia": "The capital of Australia is Canberra.",
    "what is the capital of brazil": "The capital of Brazil is Brasília.",
    "what is the capital of canada": "The capital of Canada is Ottawa.",
    "what is the capital of china": "The capital of China is Beijing.",
    "what is the capital of france": "The capital of France is Paris.",
    "what is the capital of germany": "The capital of Germany is Berlin.",
    "what is the capital of india": "The capital of India is New Delhi.",
    "what is the capital of indonesia": "The capital of Indonesia is Jakarta.",
    "what is the capital of italy": "The capital of Italy is Rome.",
    "what is the capital of japan": "The capital of Japan is Tokyo.",
    "what is the capital of mexico": "The capital of Mexico is Mexico City.",
    "what is the capital of nepal": "The capital of Nepal is Kathmandu.",
    "what is the capital of netherlands": "The capital of the Netherlands is Amsterdam.",
    "what is the capital of pakistan": "The capital of Pakistan is Islamabad.",
    "what is the capital of russia": "The capital of Russia is Moscow.",
    "what is the capital of saudi arabia": "The capital of Saudi Arabia is Riyadh.",
    "what is the capital of south africa": "South Africa has three capitals: Pretoria, Bloemfontein, and Cape Town.",
    "what is the capital of south korea": "The capital of South Korea is Seoul.",
    "what is the capital of spain": "The capital of Spain is Madrid.",
    "what is the capital of sri lanka": "The capital of Sri Lanka is Sri Jayawardenepura Kotte.",
    "what is the capital of thailand": "The capital of Thailand is Bangkok.",
    "what is the capital of turkey": "The capital of Turkey is Ankara.",
    "what is the capital of ukraine": "The capital of Ukraine is Kyiv.",
    "what is the capital of united arab emirates": "The capital of UAE is Abu Dhabi.",
    "what is the capital of united kingdom": "The capital of the United Kingdom is London.",
    "what is the capital of united states": "The capital of the United States is Washington, D.C.",
    "what is the capital of vietnam": "The capital of Vietnam is Hanoi.",
    "what is the capital of zimbabwe": "The capital of Zimbabwe is Harare.",

    # India and Neighbouring Countries - Directions and capitals
    "which countries are in the north of india": "To the north of India are China, Nepal and Bhutan.",
    "which countries are in the northwest of india": "To the northwest of India are Pakistan and Afghanistan.",
    "which countries are in the east of india": "To the east of India are Bangladesh and Myanmar.",
    "which countries are in the south east of india": "The southeastern neighbours of India include Myanmar and Indonesia.",
    "which countries are in the west of india": "To the west of India are Pakistan and Afghanistan.",
    "which countries share land borders with india": "India shares land borders with Afghanistan, Bangladesh, Bhutan, China, Myanmar, Nepal, Pakistan, and Sri Lanka.",
    "which countries share maritime borders with india": "India shares maritime borders with Sri Lanka and the Maldives.",
    "what is the capital of nepal": "The capital of Nepal is Kathmandu.",
    "what is the capital of bhutan": "The capital of Bhutan is Thimphu.",
    "what is the capital of bangladesh": "The capital of Bangladesh is Dhaka.",
    "what is the capital of myanmar": "The capital of Myanmar is Naypyidaw.",
    "what is the capital of pakistan": "The capital of Pakistan is Islamabad.",
    "what is the capital of sri lanka": "The capital of Sri Lanka is Sri Jayawardenepura Kotte.",

    # Additional info
    "what ocean is south of india": "The Indian Ocean lies to the south of India.",
    "which seas surround india": "The Arabian Sea is to the west and the Bay of Bengal is to the east of India.",
    "how many neighbours does india have": "India has 7 land neighbours and 2 maritime neighbours making 9 in total.",
}

# --- Preprocessing function for FAQ query ---
def query_faq_kb(user_input):
    # Normalize input: lowercase, strip trailing punctuations and spaces
    normalized = user_input.lower().strip().rstrip('?.!')
    return faq_knowledge_base.get(normalized, None)


# --- Load model and tokenizer ---
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

chat_history_ids = None

print("Chatbot ready! Type '/exit' to quit.\n")

while True:
    user_input = input("User: ").strip()
    if user_input.lower() == "/exit":
        print("Exiting chatbot. Goodbye!")
        break

    # Check FAQ knowledge base first
    answer = query_faq_kb(user_input)
    if answer:
        print(f"Bot: {answer}")
        # Update chat history with FAQ Q/A to maintain context consistency
        faq_turn = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
        ans_turn = tokenizer.encode(answer + tokenizer.eos_token, return_tensors='pt')
        if chat_history_ids is not None:
            chat_history_ids = torch.cat([chat_history_ids, faq_turn, ans_turn], dim=-1)
        else:
            chat_history_ids = torch.cat([faq_turn, ans_turn], dim=-1)
        continue

    # Encode user input with EOS token
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Append tokens to chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids is not None else new_user_input_ids

    # Generate response with attention mask explicitly
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        attention_mask=torch.ones(bot_input_ids.shape, dtype=torch.long)
    )

    # Decode bot reply (tokens generated after user input)
    bot_reply = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"Bot: {bot_reply}")


Chatbot ready! Type '/exit' to quit.

User: Hello
Bot: Good morning
User: Oh Hello, How are you?
Bot: I'm doing ok, what about you?
User: I'm good too
Bot: Good to hear!
User: What is the capital of Russia?
Bot: The capital of Russia is Moscow.
User: Great!, Thank you
Bot: You're welcome
User: /exit
Exiting chatbot. Goodbye!
